In [12]:
from sympy import symbols, cos, sin, diff, Function, simplify

# Define symbols
theta = symbols('theta')
k, e, h, GM, sigma = symbols('k e h GM sigma')

# Define r(theta)
r = (h ** 2 / GM) / (1 + k * e * cos(theta / sigma))

# Calculate dr/dtheta
dr_dtheta = diff(r, theta)

# Define omega(theta) = dtheta/dt
omega = h / r**2

# Calculate d^2r/dtheta^2
d2r_dtheta2 = diff(dr_dtheta, theta)

# Calculate domega/dtheta
domega_dtheta = diff(omega, theta)

dr_dt = diff(r,theta)*omega
# Calculate d^2r/dt^2 using chain rule
d2r_dt2 = simplify(d2r_dtheta2 * omega**2 + dr_dtheta * domega_dtheta * omega)

# Calculate d^2theta/dt^2
# Differentiate omega(theta)
d2theta_dt2 = simplify(domega_dtheta * omega)

# Display the results
print(f"dr/dt= {dr_dt}")
print(f"dr/dtheta = {dr_dtheta}")
print(f"d^2r/dtheta^2 = {d2r_dtheta2}")
print(f"domega/dtheta = {domega_dtheta}")
print(f"d^2r/dt^2 = {d2r_dt2}")
print(f"d^2theta/dt^2 = {d2theta_dt2}")


dr/dt= GM*e*k*sin(theta/sigma)/(h*sigma)
dr/dtheta = e*h**2*k*sin(theta/sigma)/(GM*sigma*(e*k*cos(theta/sigma) + 1)**2)
d^2r/dtheta^2 = 2*e**2*h**2*k**2*sin(theta/sigma)**2/(GM*sigma**2*(e*k*cos(theta/sigma) + 1)**3) + e*h**2*k*cos(theta/sigma)/(GM*sigma**2*(e*k*cos(theta/sigma) + 1)**2)
domega/dtheta = -2*GM**2*e*k*(e*k*cos(theta/sigma) + 1)*sin(theta/sigma)/(h**3*sigma)
d^2r/dt^2 = GM**3*e*k*(e*k*cos(theta/sigma) + 1)**2*cos(theta/sigma)/(h**4*sigma**2)
d^2theta/dt^2 = -2*GM**4*e*k*(e*k*cos(theta/sigma) + 1)**3*sin(theta/sigma)/(h**6*sigma)


In [13]:
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy import constants as cc, units as uu

# Constants
G = cc.G  # gravitational constant in m^3 kg^-1 s^-2
M = cc.M_sun  # mass of the Sun in kg
c = cc.c.si.value  # speed of light in m/s
GM = (G*M).si.value  # gravitational parameter for the Sun in m^3/s^2

# Constants for Mercury
a_mercury = 57.909e9 * uu.m  # Semi-major axis of Mercury's orbit in meters
e = e_mercury = 0.205630  # Eccentricity of Mercury's orbit
# Specific angular momentum (h) for Mercury
# h = sqrt(G * M * a * (1 - e^2))
# h_mercury = ((cc.G * cc.M_sun * a_mercury * (1 - e_mercury**2))**0.5).decompose()
# h = h_mercury.to(uu.m**2 / uu.s).value  # Convert to m^2/s for consistency with previous units used

# Calculate specific angular momentum for Mercury
h = (np.sqrt(GM * a_mercury * (1 - e**2))).si.value

In [21]:
def r_derivatives(theta, h, e, GM, x):
    sigma = x[0]
    k = x[1]
    # Expression for r, r_dot, and r_double_dot
    r = (h ** 2 / GM) / (1 + k * e * np.cos(theta / sigma))
    theta_dot = h / r ** 2
    dr_dt= GM*e*np.sin(theta/sigma)/(h*sigma)
    dr_dtheta = e*h**2*np.sin(theta/sigma)/(GM*sigma*(e*np.cos(theta/sigma) + 1)**2)
    d2r_dtheta_2 = 2*e**2*h**2**2*np.sin(theta/sigma)**2/(GM*sigma**2*(e*np.cos(theta/sigma) + 1)**3) + e*h**2*np.cos(theta/sigma)/(GM*sigma**2*(e*np.cos(theta/sigma) + 1)**2)
    domega_dtheta = -2*GM**2*e*(e*np.cos(theta/sigma) + 1)*np.sin(theta/sigma)/(h**3*sigma)
    d2r_dt2 = GM**3*e*(e*np.cos(theta/sigma) + 1)**2*np.cos(theta/sigma)/(h**4*sigma**2)
    d2theta_dt2 = -2*GM**4*e*(e*np.cos(theta/sigma) + 1)**3*np.sin(theta/sigma)/(h**6*sigma)
    return r, dr_dt, d2r_dt2, theta_dot


def newtonian_grav_accel(GM, r):
    a_theoretical = -GM / r**2
    return a_theoretical

def new_grav_accel(GM, r, r_dot, theta_dot):
    v = np.sqrt(r_dot**2 + r**2 * theta_dot**2)
    gamma_v = 1 / np.sqrt(1 - v**2 / c**2)
    B = (1 + (gamma_v - 1) * (r_dot / v)**2)
    a_theoretical = -GM / gamma_v / r**2 / B
    return a_theoretical

def error_functionHU(x):
    def integrand(theta):
        r, r_dot, r_double_dot,theta_dot = r_derivatives(theta, h, e, GM, x)
        a_theoretical = new_grav_accel(GM, r, r_dot, theta_dot)
        a_numerical = r_double_dot - r * theta_dot ** 2  # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2

    sigma = x[0]
    integral_error, _ = quad(integrand, 0, 2*np.pi/sigma, epsabs=1e-14, epsrel=1e-14)
    return 100*integral_error

def error_functionNewton(x):
    def integrand(theta):
        r, r_dot, r_double_dot,theta_dot = r_derivatives(theta, h, e, GM, x)
        a_theoretical = newtonian_grav_accel(GM, r)
        a_numerical = r_double_dot - r * theta_dot ** 2  # Assuming r_double_dot is defined to calculate \ddot{r}
        return (a_theoretical - a_numerical)**2

    sigma = x[0]
    integral_error, _ = quad(integrand, 0, 2*np.pi/sigma, epsabs=1e-14, epsrel=1e-14)
    return 100*integral_error

def calculatePrecession(errorf):
    # Initial guess for sigma
    initial_x = [0.9, 1.0]
    result = minimize(errorf, initial_x, method='Nelder-Mead', tol=1E-14)
    optimized_sigma = result.x[0]# The optimized value for sigma
    optimized_eccentricity = result.x[1]*e
    print("Optimized sigma:", optimized_sigma)
    print("Optimized sigma:", optimized_eccentricity)
    # Constants
    orbital_period_days = 88  # Orbital period of Mercury in days
    days_per_year = 365.25  # Average number of days per year, accounting for leap years
    years_per_century = 100  # Number of years in a century


    # Calculate the number of orbits Mercury completes in a century
    orbits_per_century = (days_per_year / orbital_period_days) * years_per_century
    # Calculate the precession per orbit in radians
    precession_per_orbit_radians = 2 * np.pi * (optimized_sigma - 1)

    # Convert precession per orbit to arcseconds
    precession_per_orbit_arcseconds = np.degrees(precession_per_orbit_radians) * 3600

    # Total precession in arcseconds per century
    print("HU Mercury Perihelion Precession Rate")
    total_precession_arcseconds_per_century = precession_per_orbit_arcseconds * orbits_per_century
    print(total_precession_arcseconds_per_century)

    
calculatePrecession(error_functionHU)

Optimized sigma: 1.000000086118687
Optimized sigma: 0.20562995620278152
HU Mercury Perihelion Precession Rate
46.3244161209141


In [22]:
calculatePrecession(error_functionNewton)

Optimized sigma: 0.9999999999999998
Optimized sigma: 0.2056299999999999
HU Mercury Perihelion Precession Rate
-1.194408208701506e-07


# FRAME DRAGGING CONTRIBUTION FROM SUN

In [15]:
import numpy as np

def lense_thirring_precession(G, J, c, a, e):
    """
    Calculate the Lense-Thirring precession for Mercury due to the Sun's rotation.

    Parameters:
    G (float): Gravitational constant (m^3 kg^-1 s^-2)
    J (float): Angular momentum of the Sun (kg m^2 s^-1)
    c (float): Speed of light (m/s)
    a (float): Semi-major axis of Mercury's orbit (m)
    e (float): Eccentricity of Mercury's orbit

    Returns:
    float: Lense-Thirring precession in arcseconds per century
    """
    # Calculate the Lense-Thirring precession in radians per revolution
    delta_phi_LT_rad = (2 * G * J) / (c**2 * a**3 * (1 - e**2)**(3/2))
    
    # Convert radians per revolution to arcseconds per revolution
    delta_phi_LT_arcsec = delta_phi_LT_rad * (360 * 3600) / (2 * np.pi)
    
    # Convert arcseconds per revolution to arcseconds per century
    # Mercury completes about 415 revolutions per century
    revolutions_per_century = 415
    delta_phi_LT_arcsec_per_century = delta_phi_LT_arcsec * revolutions_per_century
    
    return delta_phi_LT_arcsec_per_century

# Constants
G = 6.67430e-11  # gravitational constant in m^3 kg^-1 s^-2
c = 299792458    # speed of light in m/s
a = 57.91e9      # semi-major axis of Mercury's orbit in meters (57.91 million km)
e = 0.205630     # eccentricity of Mercury's orbit

# Angular momentum of the Sun, approximated as J = I * omega
# I for the Sun (moment of inertia) ~ 0.070 (I/MR^2) * M * R^2
# M = 1.989e30 kg (mass of the Sun), R = 696.34e6 m (radius of the Sun)
# omega = 2.865e-6 rad/s (angular velocity of the Sun, 25-day period at the equator)
M_sun = 1.989e30  # mass of the Sun in kg
R_sun = 696.34e6  # radius of the Sun in meters
omega_sun = 2.865e-6  # angular velocity of the Sun in rad/s
I_sun = 0.070 * M_sun * R_sun**2  # moment of inertia of the Sun

J_sun = I_sun * omega_sun  # angular momentum of the Sun

# Calculate Lense-Thirring Precession
delta_phi_LT = lense_thirring_precession(G, J_sun, c, a, e)
print(f"Lense-Thirring precession of Mercury's perihelion due to the Sun's rotation: {delta_phi_LT:.12f} arcseconds per century")


Lense-Thirring precession of Mercury's perihelion due to the Sun's rotation: 0.000000000135 arcseconds per century


# Black Hole

In [16]:
import numpy as np

# Constants
G = 6.67430e-11  # gravitational constant, m^3 kg^-1 s^-2
c = 299792458    # speed of light, m/s
a = 57.91e9      # semi-major axis of Mercury's orbit, m
e = 0.205630     # eccentricity of Mercury's orbit

# Black hole parameters
M_bh = 1e9 * 1.989e30  # mass of the black hole, kg (1 billion solar masses)
R_s = 2 * G * M_bh / c**2  # Schwarzschild radius of the black hole, m

# Estimating the black hole's angular velocity
# Assuming it rotates once per its own 'day', which we'll assume is 1000 seconds for calculation
# This is a purely hypothetical and illustrative assumption
T_bh = 1000  # rotational period of the black hole, seconds
omega_bh = 2 * np.pi / T_bh  # angular velocity, rad/s

# Black hole's angular momentum (J = I * omega)
# Using the simple formula for angular momentum of a rotating sphere, J = 2/5 M R^2 omega
# This is not precise for black holes but gives us a ballpark figure for illustrative purposes
I_bh = (2/5) * M_bh * R_s**2  # moment of inertia of the black hole
J_bh = I_bh * omega_bh  # angular momentum of the black hole

# Lense-Thirring Precession calculation
delta_phi_LT_rad = (2 * G * J_bh) / (c**2 * a**3 * (1 - e**2)**(1.5))
delta_phi_LT_sec = delta_phi_LT_rad * (360) / (2 * np.pi)  # convert radians to seconds
delta_phi_LT_rotations = delta_phi_LT_rad /(2 * np.pi)  # convert radians to seconds

delta_phi_LT_arcsec = delta_phi_LT_rad * (360 * 3600) / (2 * np.pi)  # convert radians to arcseconds
revolutions_per_century = 415  # Mercury completes approximately 415 revolutions per century
delta_phi_LT_arcsec_per_century = delta_phi_LT_arcsec * revolutions_per_century  # total precession per century

print(f"Lense-Thirring precession at Mercury's orbit due to a 1 billion solar mass black hole: {delta_phi_LT_rotations:.6f} rotations per year")


Lense-Thirring precession at Mercury's orbit due to a 1 billion solar mass black hole: 56.654090 rotations per year
